#### Cálculo del Intervalo de Confianza del Lift de una Métrica Ratio: **Revenue/Clics**

In [1]:
import numpy as np

# Datos simulado: 5000 usuarios.
# Cada fila es un usuario: [clics_totales, revenue_total]
np.random.seed(42)
n_users = 5000

# Grupo A (Control)
clics_A = np.random.poisson(lam=2, size=n_users) # lambda=2: Promedio 2 clics por user. Poisson es la distribución natural para conteos de eventos discretos.
revenue_A = clics_A * np.random.lognormal(mean=2, sigma=0.5, size=n_users) # La distribución Log-Normal modela el valor monetario por clic.

# Grupo B (Tratamiento) - El tratamiento mejora ambas dimensiones: cantidad de clics y revenue.
clics_B = np.random.poisson(lam=2.1, size=n_users) 
revenue_B = clics_B * np.random.lognormal(mean=2.05, sigma=0.5, size=n_users)

def bootstrap_ratio_lift(num_A, den_A, num_B, den_B, n_bootstrap=10000):
    """
    Calcula el intervalo de confianza del Lift usando Bootstrap Vectorizado.
    num = Numerador (ej. Revenue), den = Denominador (ej. Clics)
    """
    n_samples = len(num_A)
    
    # Generar índices aleatorios con reemplazo en una matriz (Vectorización), avoid using 'for' loop (very slow)
    # Forma: (n_bootstrap, n_samples) -> Matriz gigante en memoria
    idx_A = np.random.randint(0, n_samples, size=(n_bootstrap, n_samples))
    idx_B = np.random.randint(0, n_samples, size=(n_bootstrap, n_samples))

    '''
    10000 filas x 5000 columnas: 50 M enteros aleatorios, donde cada valor es un índice entre 0 y 4999.
    idx_A = [
        [2341, 0, 4999, 123, 2341, ...],  # Fila 0: Iteración bootstrap 1 → re-muestra usuario 2341, 0, 4999...
        [1, 4500, 231, 4500, 88,   ...],  # Fila 1: Iteración bootstrap 2 → re-muestra usuario 1, 4500...
        ...                               # 10,000 filas en total
        ]
    "Con reemplazo" = un mismo usuario puede aparecer varias veces en la misma fila 
    (ej: usuario 2341 aparece 2 veces en fila 0). Esto es esencial en Bootstrap.

    '''
    
    # Extraer las muestras y sumar por fila (axis=1)
    sum_num_A = np.sum(num_A[idx_A], axis=1)
    sum_den_A = np.sum(den_A[idx_A], axis=1)
    
    sum_num_B = np.sum(num_B[idx_B], axis=1)
    sum_den_B = np.sum(den_B[idx_B], axis=1)
    
    '''
    sum_num_A = np.sum(num_A[idx_A], axis=1)
                      ↑      ↑        ↑
                   revenue  índices  suma por fila
    
    num_A = [r0, r1, r2, r3, ..., r4999]  # Revenue de 5000 usuarios

    num_A[idx_A] →  # Matriz 10,000 x 5,000 de revenues re-sampleados
    [
        [r2341, r0, r4999, r123, r2341, ...],  # Bootstrap 1
        [r1, r4500, r231, r4500, r88,   ...],  # Bootstrap 2
        ...

        np.sum(..., axis=1) suma cada fila → resultado: array de 10,000 sumas:

        sum_num_A = [suma_bootstrap_1, suma_bootstrap_2, ..., suma_bootstrap_10000]
        Shape: (10000,)
    ]

    Ambos usan idx_A porque revenue y clics pertenecen al mismo usuario. Si usaran índices distintos, 
    estarías mezclando el revenue del usuario 42 con los clics del usuario 789 — eso rompería la relación 
    entre las variables.
    '''


    # Calcular los ratios (Métricas globales por cada iteración)
    ratio_A = sum_num_A / sum_den_A
    ratio_B = sum_num_B / sum_den_B
    
    # Calcular la diferencia relativa (Lift)
    lift = (ratio_B - ratio_A) / ratio_A
    
    # Obtener percentiles para el intervalo de confianza del 95%
    ci_lower = np.percentile(lift, 2.5)
    ci_upper = np.percentile(lift, 97.5)
    p_value_approx = np.mean(lift <= 0) * 2 # Test de dos colas
    
    return np.mean(lift), ci_lower, ci_upper, p_value_approx

# Ejecutar el análisis
mean_lift, lower, upper, p_val = bootstrap_ratio_lift(revenue_A, clics_A, revenue_B, clics_B)

print(f"Lift Promedio (Revenue/Clic): {mean_lift:.2%}")
print(f"Intervalo de Confianza (95%): [{lower:.2%}, {upper:.2%}]")
print(f"P-valor aproximado: {p_val:.4f}")

if lower > 0:
    print(">> Decisión: El Tratamiento B es significativamente superior.")
else:
    print(">> Decisión: No hay diferencia estadísticamente significativa.")

Lift Promedio (Revenue/Clic): 5.94%
Intervalo de Confianza (95%): [3.23%, 8.73%]
P-valor aproximado: 0.0000
>> Decisión: El Tratamiento B es significativamente superior.


- El Tratamiento B genera **5.94% más revenue por clic** que el Control A, promediado sobre las 10,000 iteraciones bootstrap.

         2.5%                       97.5%
           ↓                           ↓
──────────[3.23% ▓▓▓▓▓▓▓▓▓▓▓▓▓ 8.73%]──────────
                      ↑
                   5.94% (media)

El 95% de los escenarios posibles el lift está entre 3.23% y 8.73%

ci_lower = 3.23%	Peor escenario probable:	B sigue siendo mejor que A
ci_upper = 8.73%	Mejor escenario probable:	B podría ser aún mejor

**Clave:** El intervalo no cruza el 0%. Incluso en el peor escenario probable (+3.23%), B sigue siendo superior.

**p=0.0000** no significa probabilidad exactamente cero, sino que en ninguna de las 10,000 simulaciones B fue peor que A. Con más iteraciones (ej: 1,000,000) podría aparecer algún caso.

***¿Vale la pena implementar el Tratamiento B?:*** Sí, con alta confianza estadística.B genera entre 3.23% y 8.73% más revenue por clic que A, y esto nunca fue atribuible al azar en ninguna de las 10,000 simulaciones bootstrap.